In [ ]:
%load_ext autoreload
%autoreload 2

In [18]:
import community as community_louvain
import collections
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
from torch import Tensor
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

In [6]:
#l2g related imports
from l2gv2.clustering import distributed_clustering
from l2gv2.network.tgraph import TGraph
from l2gv2.network.npgraph import NPGraph
from l2gv2.patch.patches import create_patch_data
from l2gv2.patch.clustering import hierarchical_clustering


# <font color="grey"> Hierarchical construction of Patch Graphs</font>

###  <a id='chapter1'> <font color="grey">1. The local2global approach </font></a>

The initial step in creating the patch graph consists of clustering the graph. The clusters 

The Cora dataset is a well-known dataset in the field of graph research. This consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

In [11]:
dataset = Planetoid(root=data_dir, name='Cora')
data = dataset[0]

Processing...
Done!


In [15]:
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')

Number of nodes: 2708
Number of edges: 10556
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Average node degree: 3.90


In [16]:
G = to_networkx(data, to_undirected=True)
degrees = [val for (node, val) in G.degree()]
display(pd.DataFrame(pd.Series(degrees).describe()).transpose().round(2))
print(len(degrees))
print(sum(degrees))
plt.figure(figsize=(10, 6))
plt.hist(degrees, bins=50)
plt.xlabel("node degree")
plt.show()

NameError: name 'pd' is not defined

In [14]:
edge_index = data.edge_index.numpy()
print(edge_index.shape)

(2, 10556)
